In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import random
import os
from sklearn.cross_validation import train_test_split
import sys
import cv2

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gzip
import os
import sys

import tensorflow.python.platform

from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import itertools

class ConvNet(object): pass

def init_weights(shape, st_dev):
    return tf.Variable(tf.random_normal(shape, stddev = st_dev))
  
def init_bias(shape, st_dev):
    bias = tf.Variable(tf.random_normal(shape, stddev=st_dev))
    return (bias)

def model(X, w_layer_1, w_layer_2, w_layer_3,bias_1,bias_2,bias_3, p_keep_input = 1.0, p_keep_hidden = 1.0):
    X = tf.nn.dropout(X, p_keep_input)
    hidden_1 = tf.nn.relu(tf.add(tf.matmul(X, w_layer_1),bias_1))

    hidden_1 = tf.nn.dropout(hidden_1, p_keep_hidden)
    hidden_2 = tf.nn.relu(tf.add(tf.matmul(hidden_1, w_layer_2),bias_2))

    hidden_2 = tf.nn.dropout(hidden_2, p_keep_hidden)

    return tf.add(tf.matmul(hidden_2, w_layer_3),bias_3)

def convert_label(result):
    resultx = []
    for res in result:
        restmp = []
        if res == 'male':
            restmp = [0,1]
        elif res == 'female':
            restmp = [1,0]
        resultx.append(restmp)
    return resultx



In [2]:
IMAGE_SIZE = 30
def resize_image(image, height = IMAGE_SIZE, width = IMAGE_SIZE):
    top, bottom, left, right = (0, 0, 0, 0)
    
    #获取图像尺寸
    h, w, _ = image.shape
    
    #对于长宽不相等的图片，找到最长的一边
    longest_edge = max(h, w)    
    
    #计算短边需要增加多上像素宽度使其与长边等长
    if h < longest_edge:
        dh = longest_edge - h
        top = dh // 2
        bottom = dh - top
    elif w < longest_edge:
        dw = longest_edge - w
        left = dw // 2
        right = dw - left
    else:
        pass 
    
    #RGB颜色
    BLACK = [0, 0, 0]
    
    #给图像增加边界，是图片长、宽等长，cv2.BORDER_CONSTANT指定边界颜色由value指定
    constant = cv2.copyMakeBorder(image, top , bottom, left, right, cv2.BORDER_CONSTANT, value = BLACK)
    
    #调整图像大小并返回
    return cv2.resize(constant, (height, width))

In [3]:
def read_path(path_name):
    images = []
    labels = []
    for dir_item in os.listdir(path_name):
        #从初始路径开始叠加，合并成可识别的操作路径
        full_path = os.path.abspath(os.path.join(path_name, dir_item))
        
        if os.path.isdir(full_path):    #如果是文件夹，继续递归调用
            read_path(full_path)
        else:   #文件
            if dir_item.endswith('.jpg'):
                image = cv2.imread(full_path)
                image = resize_image(image, IMAGE_SIZE, IMAGE_SIZE)
                
                #图片变成灰度图
                #cv2.imwrite('1.jpg', image)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY );
                #xd,image = cv2.threshold(image,127,255,cv2.THRESH_BINARY)
                imagex = []
                for img in image:
                    imagex.extend(img)
                images.append(imagex)                
                labels.append(path_name)                                
                    
    return images,labels

In [4]:
def load_dataset(path_name):
    images,labels = read_path(path_name)    
  
    labels = [[0,1] if label.endswith('camera') else [1,0] for label in labels]
   
    return images, labels

In [5]:
images, labels = load_dataset("/dbfs/FileStore/tables/images/category/camera")
imagesx, labelsx = load_dataset("/dbfs/FileStore/tables/images/category/lotus")
images.extend(imagesx)
labels.extend(labelsx)

In [6]:
train_images, valid_images, train_labels, valid_labels = train_test_split(images, labels, test_size = 0.1, random_state = random.randint(0, 100)) 

In [7]:
BATCH_SIZE = 30
num_epochs = 250
train_size = len(train_labels)

In [8]:
def image_func(base_learning_rate = 0.0005,decay_rate = 0.9,conv1_size=1200, fc1_size=400):
    
#     training_epochs = 50
#     learning_rate = 0.0005
#     decay_rate = 0.9
#     momentum=0.001
    
    X = tf.placeholder("float", [None, 900])
    Y = tf.placeholder("float", [None, 2])


    w_layer_1 = init_weights([900, conv1_size], st_dev=0.01)
    bias_1 = init_bias(shape=[conv1_size], st_dev=0.01)

    w_layer_2 = init_weights([conv1_size, fc1_size], st_dev=0.01)
    bias_2 = init_bias(shape=[fc1_size], st_dev=0.01)

    w_layer_3 = init_weights([fc1_size, 2], st_dev=0.01)
    bias_3 = init_bias(shape=[2], st_dev=0.01)

    p_keep_input = tf.placeholder("float")

    p_keep_hidden = tf.placeholder("float")

    # 构建模型
    py_x = model(X, w_layer_1, w_layer_2, w_layer_3,bias_1,bias_2,bias_3, p_keep_input, p_keep_hidden)
    
    batch = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(
      base_learning_rate,                # Base learning rate.
      batch * BATCH_SIZE,  # Current index into the dataset.
      train_size,          # Decay step.
      decay_rate,                # Decay rate.
      staircase=True)
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=py_x, labels=Y))
    train_op = tf.train.RMSPropOptimizer(learning_rate, decay_rate, 0.001).minimize(cost)
    predict_op = tf.argmax(py_x, 1)
    
    res = ConvNet()
    res.prediction = tf.argmax(py_x, 1)
    res.optimizer = train_op
    res.loss = cost
    res.p_keep_input = p_keep_input
    res.p_keep_hidden = p_keep_hidden
    res.decay_rate = decay_rate
    res.learning_rate = learning_rate
    res.train_data_node = X
    res.train_labels_node = Y
    return res

In [9]:
base_learning_rates = [float(x) for x in np.logspace(-5, -3, num=10, base=10.0)]
decay_rates = [0.9,0.95]
#conv1_sizes = [16,32,64]
fc1_sizes = [100,200,400,800,1000]
all_experiments = list(itertools.product(base_learning_rates, decay_rates, fc1_sizes))
print(len(all_experiments))

In [10]:
train_data_bc = sc.broadcast(train_images)
train_labels_bc = sc.broadcast(train_labels)

def run(base_learning_rate, decay_rate, fc1_size):
    train_data = train_data_bc.value
    train_labels = train_labels_bc.value
    res = {}
    res['base_learning_rate'] = base_learning_rate
    res['decay_rate'] = decay_rate
    res['fc1_size'] = fc1_size
    res['accuracy'] = 0.0
    res['validation_accuracy'] = 0.0
    
    with tf.Session() as s:
        # Create the computation graph
        graph = image_func(base_learning_rate, decay_rate, fc1_size=fc1_size)
        # Run all the initializers to prepare the trainable parameters.
        tf.global_variables_initializer().run()
        # Loop through training steps.
#         for step in xrange(num_epochs * train_size // BATCH_SIZE):
#             # Compute the offset of the current minibatch in the data.
#             # Note that we could use better randomization across epochs.
#             offset = (step * BATCH_SIZE) % (train_size - BATCH_SIZE)
#             batch_data = train_data[offset:(offset + BATCH_SIZE)]
#             batch_labels = train_labels[offset:(offset + BATCH_SIZE)]
#             feed_dict = {graph.train_data_node: batch_data,
#                            graph.train_labels_node: batch_labels}
#             s.run(graph.optimizer,feed_dict=feed_dict)
        for i in range(num_epochs):
            for start, end in zip(range(0, len(train_labels), BATCH_SIZE), range(BATCH_SIZE, len(train_labels), BATCH_SIZE)):
                s.run(graph.optimizer, feed_dict = {graph.train_data_node: train_images[start:end], graph.train_labels_node: train_labels[start:end], graph.p_keep_input: 0.8, graph.p_keep_hidden: 0.5})
        res['accuracy'] = np.mean(np.argmax(valid_labels, axis = 1) == s.run(graph.prediction, 
                          feed_dict = {graph.train_data_node: valid_images, graph.train_labels_node: valid_labels, graph.p_keep_input: 1.0, graph.p_keep_hidden: 1.0}))
        res['validation_accuracy'] = np.mean(np.argmax(train_labels, axis = 1) == s.run(graph.prediction, 
                          feed_dict = {graph.train_data_node: train_images, graph.train_labels_node: train_labels, graph.p_keep_input: 1.0, graph.p_keep_hidden: 1.0}))
        
        return res

In [11]:
run(0.0005,0.9,200)

In [12]:
num_nodes = 4
n = max(2, int(len(all_experiments) // num_nodes))
grouped_experiments = [all_experiments[i:i+n] for i in range(0, len(all_experiments), n)]
all_exps_rdd = sc.parallelize(grouped_experiments, numSlices=len(grouped_experiments))
results = all_exps_rdd.flatMap(lambda z: [run(*y) for y in z]).collect()

In [13]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [14]:
df = pd.DataFrame(results)
df

In [15]:
fig,ax = plt.subplots()
for fc1_size in fc1_sizes:
    #ax = df[(df.decay_rate == 0.90) & (df.fc1_size == fc1_size)].plot(x='base_learning_rate', y='accuracy')
    ax.plot(df[(df.decay_rate == 0.95) & (df.fc1_size == fc1_size)]["base_learning_rate"],df[(df.decay_rate == 0.90) & (df.fc1_size == fc1_size)]["accuracy"])
lines = ax.get_lines()
ax.legend(lines, [str(x) for x in fc1_sizes], loc='upper right')
ax.set_ylabel("Test Accuracy")
ax.set_xlabel("Base learning rate")
display()